#1 - Tubería

##1.0 - Librerías

In [ ]:
import pathlib
import os
import sklearn
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup
import urllib.request
import re
import datetime as dt
# Instalación de la API
#!pip install git+https://github.com/tweepy/tweepy.git
#import tweepy
from tqdm import tqdm
!pip install emoji_extractor
from emoji_extractor.extract import Extractor
!pip install clean-text
from cleantext import clean
!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.
##import torch
from transformers import BertForMaskedLM, BertTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for emoji-extractor: filename=emoji_extractor-1.0.20-py3-none-any.whl size=63580 sha256=9c06918564708cd73d7c534ca7a7fa967c7e00a477ce4d2eeae4d1d748f03d4b
  Stored in directory: /root/.cache/pip/wheels/c9/3a/10/4d60270bd4c7b1569ee392d611fbcbefc2af7d7245163913e3
Successfully built emoji-extractor
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=8467dd4372a82473882116eca8d8bda62ae064f7a7996314170cee923e94824e
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
     |████████████████████████████████| 120 kB 67.4 MB/s 
--2022-09-06 21:40:57--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.211
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409871727 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz’

pytorch_weights.tar 100%[===================>] 390.88M  8.25MB/s    in 50s     

2022-09-06 21:41:48 (7.81 MB/s) - ‘pytorch_weights.tar.gz’ saved [409871727/409871727]

--2022-09-06 21:41:49--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.2

##1.3 - Tweets
Descarga de las publicaciones (parte de la extracción) y transformación.

Generación del marco de datos o dataset. Se incluyen los siguientes campos: nombre, usuario, ubicación (si lo tiene), descripción del usuario, verificación de la cuenta, fecha, texto, hashtags, y fuente (dispositivo) del tweet.

In [ ]:
dict_val = {'accentos':1, 'mayusculas':1, 'sign_punt':1, 'emojis':-1, 'abrev':-1, 'extranjerismos': -1, 'repet_termin':-1, 'beto':0 }
df_val = pd.DataFrame([[key, dict_val[key]] for key in dict_val.keys()], columns=['VALORACIONES', 'Valor'])
df_val

,VALORACIONES,Valor
0,accentos,1
1,mayusculas,1
2,sign_punt,1
3,emojis,-1
4,abrev,-1
5,extranjerismos,-1
6,repet_termin,-1
7,beto,0


---
# 2 - Modelo

### 2.1 - Preprocesado
Eliminación de las palabras sin sentido o *stop words* y agrupación de emojis al final de la cadena.

**¡OJO! Aquellas que contengan acentos o estén mal escritas, se quedarán.**

In [ ]:
# Textos de prueba
NA_1 = "Esta prueba consiste en quedarnos únicamente con las palabras y eliminar aquellas sin significado. "
NA_2 = 'En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. '
NA_3 = 'Árbol de la familia de las ebenáceas, de diez a doce metros de altura y de copa ancha.'
NA_4 = 'Mi tío tiene un apartamento con mucha luz y por la ventana se puede ver el mar.'
NA_5 = 'El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera.'
NA_6 = 'Juego de mesa entre dos personas que se practica sobre un damero en el que se disponen las 16 piezas de cada jugador, desiguales en importancia y valor, que se desplazan y comen las del contrario según ciertas reglas.'
NA_7 = 'Red informática mundial, descentralizada, formada por la conexión directa entre computadoras mediante un protocolo especial de comunicación.'
NA_8 = 'Instrumento astronómico usado antiguamente para determinar la posición de los astros.'
NA_9 = 'Cuando los días pasan y las horas transcurren te das cuenta de que la soledad no es una buena amiga.'
NA_10 = 'Arbusto cultivado originario de Asia, de la familia de las rutáceas, de unos dos metros de altura, ramoso, con hojas casi persistentes, opuestas, aovadas, lisas y lustrosas, flores pequeñas en racimo, blancas y olorosas y por frutos bayas rojas, redondas y del tamaño de un guisante 😺 .'

# Textos de prueba
NB_1 = "coger el resto... Prueba, lol . Pedro, nadies hindu tkm "
NB_2 = 'Es una obra de arte porque el arte es asi'
NB_3 = 'La prueba ya con más cositas como 😍😍😺😺👏👏👏. \n A ver si funciona 😍😍😺😺, LOL'
NB_4 = 'tkm, estao ha de salir mal sino no sirve 😍😍😍.'
NB_5 = 'El arbol tene frutas buenas😺, plz'
NB_6 = 'mi amiga se fue corriendo a su casa para la inaguración'
NB_7 = 'Estoy empezando a estudiar Francés rt, omg'
NB_8 = 'Las nuevas generaciones crecen muy rapido, tan rapido como nosotros.'
NB_9 = 'la casa de mis padres es grande y tiene una picina que comprastes'
NB_10 = 'Los anacardos de volor verde aun no estan listos para ser recogidos'



# Creación del DataFrame para probar, no será necesario porque vendrá de la tubería
dicc = {'name': ['NA_1', 'NA_2', 'NA_3', 'NA_4', 'NA_5', 'NA_6', 'NA_7', 'NA_8', 'NA_9', 'NA_10', 'NB_1', 'NB_2', 'NB_3', 'NB_4', 'NB_5', 'NB_6', 'NB_7', 'NB_8', 'NB_9', 'NB_10'], 
        'text': [NA_1, NA_2, NA_3, NA_4, NA_5, NA_6, NA_7, NA_8, NA_9, NA_10, NB_1, NB_2, NB_3, NB_4, NB_5, NB_6, NB_7, NB_8, NB_9, NB_10]}
df_demo = pd.DataFrame(dicc)
df_demo

,name,text
0,NA_1,Esta prueba consiste en quedarnos únicamente c...
1,NA_2,"En un lugar de la Mancha, de cuyo nombre no qu..."
2,NA_3,"Árbol de la familia de las ebenáceas, de diez ..."
3,NA_4,Mi tío tiene un apartamento con mucha luz y po...
4,NA_5,"El resto della concluían sayo de velarte, calz..."
5,NA_6,Juego de mesa entre dos personas que se practi...
6,NA_7,"Red informática mundial, descentralizada, form..."
7,NA_8,Instrumento astronómico usado antiguamente par...
8,NA_9,Cuando los días pasan y las horas transcurren ...
9,NA_10,"Arbusto cultivado originario de Asia, de la fa..."


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
# Diccionario español
stops = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Limpieza de datos
# Movemos emojis al final y removemos espacios de más
def clean_df(df):

  extract = Extractor()
  aux = df['text'].apply(lambda x: " ".join(extract.count_emoji(x).keys() ))
  aux2 = df['text'].apply(lambda x: clean( x, 
                                              lower=False,no_emoji=True,
                                              no_line_breaks=True,
                                              fix_unicode=False,
                                              to_ascii=False
                                          )
                )
  aux = aux2+" "+aux

  df['text_clean'] = aux
  return df

df_demo = clean_df(df_demo)
df_demo

,name,text,text_clean
0,NA_1,Esta prueba consiste en quedarnos únicamente c...,Esta prueba consiste en quedarnos únicamente c...
1,NA_2,"En un lugar de la Mancha, de cuyo nombre no qu...","En un lugar de la Mancha, de cuyo nombre no qu..."
2,NA_3,"Árbol de la familia de las ebenáceas, de diez ...","Árbol de la familia de las ebenáceas, de diez ..."
3,NA_4,Mi tío tiene un apartamento con mucha luz y po...,Mi tío tiene un apartamento con mucha luz y po...
4,NA_5,"El resto della concluían sayo de velarte, calz...","El resto della concluían sayo de velarte, calz..."
5,NA_6,Juego de mesa entre dos personas que se practi...,Juego de mesa entre dos personas que se practi...
6,NA_7,"Red informática mundial, descentralizada, form...","Red informática mundial, descentralizada, form..."
7,NA_8,Instrumento astronómico usado antiguamente par...,Instrumento astronómico usado antiguamente par...
8,NA_9,Cuando los días pasan y las horas transcurren ...,Cuando los días pasan y las horas transcurren ...
9,NA_10,"Arbusto cultivado originario de Asia, de la fa...","Arbusto cultivado originario de Asia, de la fa..."


In [ ]:
def remove_stopwords_(tweet):
  df_demo['stop_words'] = df_demo['text_clean']\
                        .apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))
  return(df_demo)

df_demo = remove_stopwords_(df_demo)

## 2.2 - Valoraciones
Cada una de las próximas iteraciones representa una columna con una puntuación por cada usuario o fila. 

El sentido funcional de las valoraciones está basado de forma relativa, es decir, una proporción (modelo no supervisado). 

El sentido operacional es similar al de un modelo de one-hot, donde finalmente, una fila sumará la puntuación obtenida en cada columna o iteración.

El resultado será una agrupación por puntuaciones obtenidas, pudiéndose agrupar tanto a nivel vertical como horizontal.



### 2.2.1 - Mayúsculas
Numerador: nº de mayúsculas.

Denominador: nº de palabras (sin stop_words, igual para el resto).

In [ ]:
def get_capital_letters_(tweet):
	result = re.findall(r'[A-Z]',tweet)
	string = "".join(result)
	return list(string)
# Nueva columna pasando la función por cada una de las filas	
df_demo['capital_letters'] = df_demo['stop_words'].apply(lambda x : len(get_capital_letters_(x))/len(x.split()))

### 2.2.2 - Acentos
Numerador: nº de palabras con acentos.

Denominador: nº de palabras.

In [ ]:
def find_accent_words(tweet):
  acentos = ["áéíóúüÁÉÍÓÚÜ"]
  words = np.array([])
  for pat in acentos[0]:
    aux = "\w*["+pat+"]\w*"
    aux = re.findall(aux, tweet)
    if len(aux) > 0:
      words = np.append(words, aux)
  return(len(words)/len(tweet.split()))

df_demo['accents'] = df_demo['stop_words'].apply(lambda x: find_accent_words(x))

### 2.2.3 - Signos de Puntuación
Numerador: nº de signos.

Denominador: nº de caracteres.

In [ ]:
def get_punctuations_marks_(tweet):
	result = re.findall(r'[!"\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]*',
						tweet)
	string = "".join(result)
	return list(string)
# Nueva columna pasando la función por cada una de las filas	
df_demo['punctuation_marks'] = df_demo['stop_words'].apply(lambda x : len(get_punctuations_marks_(x))/len(x))

### 2.2.4 - Longitud de Palabra
Numerador: suma de los caracteres de todas las palabras.

Denominador: nº de palabras.

In [ ]:
# Longitud de cada palabra -> String de números
def word_length(tweet):
  aux = [len(x) for x in tweet.split()]
  return aux

# Suma de los números de cada string para luego relativizarlo entre el número de palabras
def sum_numbers(numbers):
     if len(numbers) == 0:
         return 0
     return numbers[0] + sum_numbers(numbers[1:])

df_demo['word_length'] = df_demo['stop_words'].apply(lambda x: sum_numbers(word_length(x))/len(x.split()))

### 2.2.5 - Emojis
Numerador: n° de emojis únicos.

Denominador: n° de palabras.

In [ ]:
def detect_emoji(tweet):
  extract = Extractor()
  return len(extract.count_emoji(tweet).keys())/len(tweet.split())

df_demo['emoji'] = df_demo['stop_words'].apply(lambda x: detect_emoji(x))

### 2.2.6 - Abreviaciones
Numerador: nº de abreviaciones.

Denominador: nº de palabras.

In [ ]:
def get_abbreviation_(tweet):
  abbr = ['rofl','pa','rl','lol','ptm','tqm','tkm','bb',
         'fb','gpi','yt','tl','pls','oc','wtf','bc','bfn',
         'dm','ftf','icymi','imho','irl','lmlt','np','oh',
         'plz','qotd','rt','prt','rtrl','tmi','ty','tt','tl',
         'cc','fa','fyi','+1','afk','brb','gtg','ht','hth', 'imo','lmao']
  words = np.array([])
  for pat in abbr[:]:
    aux = " "+pat+" "
    aux = re.findall(aux, tweet)
    if len(aux) > 0:
      words = np.append(words, aux)
  return (len(words)/len(tweet.split()))
  
df_demo['abbreviation'] = df_demo['stop_words'].apply(lambda x: get_abbreviation_(x))

### 2.2.7 - Barbarismos
Numerador: nº de extranjerismos.

Denominador: nº de palabras.

In [ ]:
def get_barbarism_(tweet):
  barbar = ['comprastes','guevo','inaguración','nadies','picsa','custión',
          'interperie','fuistes','ambos dos','jrito','hebreo','vertir',
          'hindú','trompezar','adición','exepto','lego','líbido','hubiero',
          'idiosincracia','beneficiencia','visicitud','suscinto','aférrimo',
          'excéptico','convalescencia','discrección','esplanada','innundación',
          'fideligno','fregaplatos','inexcrutable','misógeno','prevadicación',
          'subrealista','sujección','transtornado','exalar','exhuberante',
          'exumar','exausto','exibir']
  words = np.array([])
  for pat in barbar[:]:
    aux = "\w*"+pat+"\w*"
    aux = re.findall(aux, tweet)
    if len(aux) > 0:
      words = np.append(words, aux)
  return (len(words)/len(tweet.split()))
  
df_demo['barbarism'] = df_demo['stop_words'].apply(lambda x: get_barbarism_(x))

### 2.2.8 - Repetición de Términos
Numerador: n° de palabras únicas.

Denominador: n° de palabras totales.

In [ ]:
def repeated_words(tweet):
  aux = tweet.split()
  aux_unique = np.unique(aux)
  return(1 - len(aux_unique)/len(aux))
  
df_demo['repeticion'] = df_demo['stop_words'].apply(lambda x: repeated_words(x))

### 2.2.9 - BETO
*Comparación con respuestas del BETO*.

Numerador: N° de palabras coincidentes con las primeras 10 propuestas del BETO.

Denominador: N° de palabras.

In [ ]:
def beto_pred(tweet):
  split_tweet = re.findall(r"[\w']+|[.,¡!¿?;:]", tweet)
  c = 0

  
  for i in range(len(split_tweet)):
    tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)
    model = BertForMaskedLM.from_pretrained("pytorch/")
    e = model.eval()
    # Ciclo por cada palabra o signo de interrogación del tweet
    text = split_tweet.copy()
    # Copia de la palabra a buscar.
    real_word = text[i]
    # Enmascarado de la palabra a buscar
    text[i] = "[MASK]"
    text = "[CLS] "+" ".join(text)+" [SEP]"
    
    # Tokenización del Tweet
    tokens = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    tokens_tensor = torch.tensor([indexed_tokens])

    # Predicción del Tweet
    predictions = model(tokens_tensor)[0]

    idxs = torch.argsort(predictions[0, i+1], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:2])

    # print(text)
    # print(predicted_token)
    c = c + (real_word in predicted_token)
  return c/len(split_tweet)
    

# beto_pred("¿hola como estas? Espero no del todo mal. Aún no recibo la postal. Que prometiste el día en que te fuiste.")

df_demo['BETO'] = df_demo['text_clean'].apply(lambda x: beto_pred(x))

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: ignored

In [ ]:
df_demo

---
## 2.3 - Segmentación / Clustering
Agrupación de las valoraciones por segmentos o clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
df_val = df_demo.iloc[:,4:] # Selecionar filas de valoraciones
array_val = df_val.to_numpy()# Convertir a array
scaler = StandardScaler()#Escalamos los valores de l'array
scaled_val = scaler.fit_transform(array_val)
df_val

In [ ]:
# K-means con 2 clusters(mayor/menot nivel de educación)
kmeans = KMeans(
  init="random",
  n_clusters=2,
  n_init=10,
  max_iter=300,
  random_state=42
)

kmeans.fit(scaled_val)

# Categoria de cada tweet en el cluster
kmeans_label = kmeans.labels_[:]
label_val = list(kmeans_label)
# Adjuntar valores (label) al df
df_demo.loc[:,'Label'] = (label_val)
df_final = df_demo.iloc[:,[0,4,5,6,7,8,9,10,11]]
df_final

In [ ]:
# Localización de los centroides
kmeans.cluster_centers_

###PCA

In [ ]:
# PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduc_val = pca.fit_transform(scaled_val)
reduc_val

In [ ]:
pca_reduc_df = pd.DataFrame(data=reduc_val, columns = ['Componente_1', 'Componente_2'])
pca_label_df = pd.concat([pca_reduc_df, df_demo[['Label']]], axis=1)

pca_label_df

In [ ]:
fig = plt.figure(figsize = (6,6))

ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Componente 1', fontsize = 15)
ax.set_ylabel('Componente 2', fontsize = 15)
ax.set_title('Componentes Principales', fontsize = 20)

color_theme = np.array(["blue", "green", "orange"])
ax.scatter(x = pca_reduc_df.Componente_1, y = pca_label_df.Componente_2,
           c=color_theme[pca_label_df.Label], s=10)
#plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', c='black', s=20)
plt.show()